In [137]:
from PIL import Image
from io import BytesIO
import glob, os, requests, base64, shutil, json

In [138]:
# the photos I want to work with
images_dir = f"{os.environ['HOME']}/Desktop/RandomPhotos/MediumExample/"

# API Endpoint for SN20 / BitAgent
api_endpoint = "https://api.gogoagent.ai/query_subnet"

# TODO USE YOUR API KEY
hdrs = {"Content-Type": "application/json",
        "Authorization": "YOUR API KEY GOES HERE"} # TODO use your API KEY

# Define the tools
# one for moving to animals directory and one for moving to non animals directory
move_to_animals_dir_tool = {
  "name": "move_to_animals_dir",
  "description": "Move the Image that contains something animal related to the animals directory",
  "arguments": {}
}
move_to_non_animals_dir_tool = {
  "name": "move_to_non_animals_dir",
  "description": "Move the Image that does NOT contain anything animal related to the non_animals directory",
  "arguments": {
  }
}
move_to_unknown_dir_tool = {
  "name": "move_to_unknown_dir",
  "description": """Move the Image to the unknown directory because 
  it can not be determined if it contains an animal or not""",
  "arguments": {
  }
}

def move_image_file(file, dirname):
    the_path = os.path.dirname(file)
    full_dirname = the_path + "/" + dirname + "/"
    if not os.path.exists(full_dirname):
        os.mkdir(full_dirname)
    print(f"Moving {file} to {full_dirname}")
    shutil.move(file, full_dirname)

def move_to_unknown_dir(file):
    move_image_file(file, "unknown")
def move_to_animals_dir(file):
    move_image_file(file, "animals")
def move_to_non_animals_dir(file):
    move_image_file(file, "non_animals")
    
tools = [move_to_animals_dir_tool, move_to_non_animals_dir_tool, move_to_unknown_dir_tool]

def get_details_about_image(image):
    # base64 encode the image for sending to the API
    img_str = get_img_str_from_image(image)
    
    # Setup the messages / prompt
    messages = [
      {"role": "user",
       "content": "Tell me about this image, I am most interested in knowing if it contains anything related to an animal or not"}
    ]

    files = [{'content': img_str, 'type': "Image"}]
    
    # Send the request to the API
    data = {"messages": messages, "files": files}
    result = requests.post(api_endpoint, headers=hdrs, json=data)
    
    return result.json()[0]['response']

def get_img_str_from_image(image):
    with open(image, 'rb') as im:
        img_str = base64.b64encode(im.read()).decode('utf-8')
        return img_str

def get_tool_call_based_on_response(response, tools):
    # Setup the messages for tool call
    messages = [
      {"role": "user",
       "content": "Tell me about this image, I am most interested in knowing if it contains anything related to an animal or not"},
      {"role": "assistant",
       "content": response},
      {"role": "user",
       "content": "I want to organize my photos, which tool should I use for the image details provided?"}
    ]
    
    # Send the request to the API
    data = {"messages": messages, "tools": tools}
    result = requests.post(api_endpoint, headers=hdrs, json=data)

    return result.json()[0]['response']    

def handle_agent_tool_call(agent_response, image_file):
    method_to_call = None
    for resp in json.loads(agent_response['response']):
        if resp['role'] == "tool call":
            method_to_call = resp['content']['name']
            break
    
    if method_to_call:
        globals()[method_to_call](image_file)
    else:
        print("no method to call")

In [139]:
for img_file in glob.glob(f"{images_dir}*.png"): # TODO change from png to whatever you're looking for
    agent_response_about_image = get_details_about_image(img_file)
    agent_response_about_tools = get_tool_call_based_on_response(agent_response_about_image, tools)
    handle_agent_tool_call(agent_response_about_tools, img_file)